In [29]:
using JSON, SDDP,DataFrames
using Plots, StatPlots
const mm = Plots.mm
const pt = Plots.pt
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm)
gr()

Plots.GRBackend()

In [124]:
# Plot main figure
function getresults(results, key, scalefactor=1.0)
    Q = [0.0, 0.05, 0.5, 0.95, 1.0]
    g2 = hcat([scalefactor*r[key] for r in results]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    DataFrame(week=1:size(g3, 1), x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5])
end
function plotpowder(data, key, ylims, ylabel,title,scalefactor=1.0)
    blue = getresults(data, key, scalefactor)
    
    plot()
    
    plot!(blue, :week, :x10, fill=(:x90, "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
    plot!(blue, :week, :x25, fill=(:x75, "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
    plot!(blue, :week, :x50, c="#00467F", w=2, linestyle=:dot)

    plot!(1:52, scalefactor*data[10][key], color="#00467F", linewidth=3)
    
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
#     if key == :price
#         N = 9
#         for t in 1:52
#             (minprice, maxprice) = extrema([d[:price][t] for d in data])
#             if minprice == maxprice
#                 scatter!([t], [minprice],c="red", markersize=2.5, markeralpha=0.5, markerstrokewidth=0)
#             else
#                 scatter!(fill(t, N), linspace(minprice, maxprice, N),c="red", markersize=2.5, markeralpha=0.5, markerstrokewidth=0)
#             end
#         end
#     end
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xlims=(1,52), xticks=(xticks, xticklabels), xlabel="")
end

function plotall(data)
#     splice!(data, 47)
    for d in data
        d[:contracted_proportion] = fill(0.0, 52)
        for t in 1:52
            d[:contracted_proportion][t] = sum(d[:milk_sales][1:t]) ./ sum(d[:milk]) 
        end
    end
    plot(
        plotpowder(data, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(a)"),
        plotpowder(data, :P, (1.25,3.3), "Pasture Cover\n(t/Ha)","(b)", 0.001),
        plotpowder(data, :b, (0,6), "Palm Kernel Fed\n(kg/Cow/Day)", "(c)", 1 / 3 / 7),
        plotpowder(data, :price, (2,10), "Futures Price\n(\$/kg)","(d)"),
        plotpowder(data, :M, (-1500,1500), "Unsold Milk\n(kg)", "(e)"),
        plotpowder(data, :contracted_proportion, (0, 1.5), "Milk Sales\n(kg)", "(f)", 1),
        layout=(2,3), size=(1500,600)
    )
end

plotall (generic function with 1 method)

In [128]:
idxs = [d[:contracted_proportion][40] < 1.5 for d in averse_sell];

In [130]:
neutral = SDDP.load("NeutralWithContracting.results");
plotall(neutral[idxs])
savefig("neutral_no_selling.pdf")

averse = SDDP.load("AverseNoContracting.results");
plotall(averse[idxs])
savefig("averse_no_selling.pdf")

averse_sell = SDDP.load("AverseWithContracting.results");
plotall(averse_sell[idxs])
savefig("averse_selling.pdf")

In [131]:
function plotdensity!(data, label, color, w=1, a=1.0, densityflag=true)
    x = [sim[:objective] - 3536 for sim in data]
    if densityflag
        density!(x, label=label, c=color, w=w, alpha=a)
    else
        vline!([mean(x)], label="", c=color, w=w, alpha=a, linestyle=:dash)
    end
end
plot()
plotdensity!(neutral, "Risk Neutral - No Contracts", "#00467F",3, 0.4, true)
plotdensity!(averse, "Risk Averse - No Contracts", "#00467F", 3, 0.7, true)
plotdensity!(averse_sell, "Risk Averse - With Contracts", "#00467F", 3, 1.0, true)

plotdensity!(neutral, "Risk Neutral - No Contracts", "#00467F",3, 0.4, false)
plotdensity!(averse, "Risk Averse - No Contracts", "#00467F", 3, 0.7, false)
plotdensity!(averse_sell, "Risk Averse - With Contracts", "#00467F", 3, 1.0, false)

plot!(ytick=false, xlabel="Operating Profit (\$/Ha)", ylabel="Smoothed Density")
plot!(size=(6 * 150,3 * 150), right_margin=5Plots.mm, legend=:topleft, xlims=(-4000, 8000))
savefig("density_comparison.pdf")

In [36]:
function undoformatting(x)
    if x[end] == 'K'
        return 1_000.0 * parse(Float64, String(x[1:(end-1)]))
    elseif x[end] == 'M'
        return 1_000_000.0 * parse(Float64, String(x[1:(end-1)]))
    else
        return parse(Float64, x)
    end 
end
# iteration, time, simulation, bound
function getlogdata(filename)
    y = open(filename, "r") do io
        y = Array{Float64}(0, 4)
        for i in 1:14
            readline(io)
        end
        while !eof(io)
            line = readline(io)
            if contains(line, "--")
                break
            end
            items = split(strip(line))
            if length(items) == 1
                break
            end
            z = undoformatting.(items[[4,5,1,2]])'
            y = vcat(y, z)
        end
        y
    end

    y[:, 1] = 1:size(y, 1)
    y
end

r0 = getlogdata("AverseContracting.log")
r3 = getlogdata("AverseContractingRib3.log")
r5 = getlogdata("AverseContractingRib5.log")
r9 = getlogdata("AverseContractingRib9.log");

In [73]:
plot(r9[:,1], r9[:, 2])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 10000 
 
 
 20000 
 
 
 30000 
 
 
 40000 
 
 
 50000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [70]:
N = 25
plot()
plot!(r3[1:N:size(r3, 1), 2], r3[1:N:size(r3, 1), 4], label="3 Ribs", c="#00467F", alpha=0.4, w=2)
plot!(r5[1:N:size(r5, 1), 2], r5[1:N:size(r5, 1), 4], label="5 Ribs", c="#00467F", alpha=0.7, w=2)
plot!(r9[1:N:size(r9, 1), 2], r9[1:N:size(r9, 1), 4], label="9 Ribs", c="#00467F", alpha=1.0, w=2)
plot!(r0[1:N:size(r0, 1), 2], r0[1:N:size(r0, 1), 4], label="Dynamic", c="#e65100", linestyle=:dot, w=2)
plot!(xlabel="Solution Time (s)\n", ylabel="Risk-adjusted Operating Profit\nUpper Bound (\$/Ha)")
plot!(xlims=(0, 21_600), ylims=(4000, 5000), size=(6*150, 3*150))
savefig("performance_comparison.pdf")